# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **LBOGANG kHOSA**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

In [ ]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')

****Loading the Data****

In [9]:
train  = pd.read_csv('../input/edsa-climate-change-belief-analysis-2022/train.csv')
test = pd.read_csv('../input/edsa-climate-change-belief-analysis-2022/test.csv') 
df_submission = pd.read_csv('../input/edsa-climate-change-belief-analysis-2022/sample_submission.csv')


train.head(10)

In [11]:
test.head()

In [15]:
df = pd.read_csv("../input/edsa-climate-change-belief-analysis-2022/train.csv")

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:
df.shape

# Basic Preprocessing will include:
**Removing handles,urls, numbers,hashtags,punctuation(non alpha characters),white space, converting text to lowercase, lematizing the texts and expanding text contractions.**

In [18]:
#Remove handles from the tweets
def remove_handle(text):
    twitter_handle = r'@[a-zA-Z0-9_]{0,15}'
    cleaned = re.sub(twitter_handle, ' ', text)
    return cleaned

In [19]:
#Removing hashtags from the tweets
def remove_hashtags(text):
    hashtags = r'#+[a-zA-Z0-9(_)]{1,}'
    cleaned = re.sub(hashtags, ' ', text)
    return cleaned

In [20]:
#Removing URLs from the tweets
def remove_url(text):
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    partial_url = r'\bht.[^\s]*'
    cleaned = re.sub(pattern_url, '', text)
    cleaned = re.sub(partial_url, '', text)
    return cleaned

In [21]:
#Removing numbers from the tweets
def remove_numbers(text):
    numbers = r'\w*\d\w*'
    cleaned = re.sub(numbers, ' ', text)
    return cleaned

In [22]:
#converting tweets to lowercase
def to_lowercase(text):
    cleaned = text.lower()
    return cleaned

In [23]:
#Removing non alpha numeric characters from the tweets
def remove_non_alpha(text):
    non_alpha = r'[^a-zA-Z\d\s]'
    cleaned = re.sub(non_alpha, ' ', text)
    return cleaned

In [24]:
#Removing the Stopwords from the text, In this case only 'rt'
def remove_stopwords(text):
    stop_words = ['rt']

    filtered_sentence = ' '.join([word for word in text.split() if word not in stop_words])
    return filtered_sentence

In [25]:
#Removing extra whitespaces from the text
def remove_whitespace(text):
    whitespace = r'\s+'
    cleaned = re.sub(whitespace, ' ', text)
    return cleaned

In [26]:
#Removing random single text characters from the text
def remove_single_characters(text):
    single_character = r'\b[a-zA-Z]\b'
    cleaned = re.sub(single_character, ' ', text)
    return cleaned

In [27]:
#Expand contraction in the text
def expand_contractions(text):
    '''
    Expand contraction in the text.
    '''
    text_list = text.split()
    expanded_list = []
    for word in text_list:
        expanded_list.append(contractions.fix(word))
        
    new_text = ' '.join(expanded_list)

    return new_text

In [28]:
def get_tokens(text):
    return text.split()

In [29]:
def lemmatize_text(text):
    '''
    Lemmatize the text.
    '''
    lemmatizer  = WordNetLemmatizer()
    lemmatized_sentence = ([lemmatizer.lemmatize(w) for w in text])
    return lemmatized_sentence

In [30]:
import re

#Cleaning dataset
def preprocess(file_import, stemming=False, lemmatize=False):
    if file_import == 'train':
        df = pd.read_csv("../input/edsa-climate-change-belief-analysis-2022/train.csv")
    elif file_import == '../input/edsa-climate-change-belief-analysis-2022/test.csv':
        df = pd.read_csv("../input/edsa-climate-change-belief-analysis-2022/test.csv")
    else:
        error = 'Incorrect File Name!!'
        return error  


    # Remove twitter handles
    df['clean_tweet'] = df['message'].apply(lambda text: remove_handle(text))
    # Remove hashtags
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_hashtags(text))
    # Remove url's
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_url(text))

    # Remove all non alphanumeric characters
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_non_alpha(text))
    # Remove numbers
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_numbers(text))
    # Set to lower case
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: to_lowercase(text))
    # Remove single characters
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_single_characters(text))
    # Remove extra whitespace
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_whitespace(text))
    # Remove stop words
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_stopwords(text))
    # Get tokens
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: get_tokens(text))
    # Lemmatize text
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: lemmatize_text(text))


    
    df['message'] = df['clean_tweet']

    df = df.drop(['clean_tweet'], 1)
    return df   

In [31]:
processed_df = preprocess('train')

In [32]:
processed_df 

# Vectorizing Training data

In [33]:
# define a function to vectorize the lists of words 
from sklearn.feature_extraction.text import TfidfVectorizer
tt = TfidfVectorizer(preprocessor=list, tokenizer=list, ngram_range=(1,2), min_df=2, strip_accents = 'ascii', smooth_idf=False)
tt.fit(processed_df['message'])
train_vect = tt.transform(processed_df['message'])


In [34]:
train_vect.toarray()

# Split the training data into features and labels.

In [35]:
X = train_vect.toarray()
y = df['sentiment']

In [36]:
X.shape

# Split the training data further into training and validation data.

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [39]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = "liblinear", random_state = 100)

# Fit the model on the training subset.

In [40]:
lr.fit(X_train,y_train)

In [41]:
lr.intercept_[0]

In [42]:
lr.coef_

# Predict on the validation subset.

In [43]:
y_pred_test = lr.predict(X_test)
y_pred_test

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_test, y_pred_test)

In [46]:
labels = [' -1', ' 0',' 1 ', ' 2']

pd.DataFrame(data=confusion_matrix(y_test, y_pred_test), index=labels, columns=labels)

# Calculate the performance metrics on the validation predictions.

In [47]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred_test, average = 'micro')

In [48]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred_test, average = 'macro')

In [ ]:
check

# Clean the test data

In [51]:
import re

#Cleaning dataset
def preprocess(file_import, stemming=False, lemmatize=False):
    if file_import == 'test':
        df = pd.read_csv("../input/edsa-climate-change-belief-analysis-2022/test.csv")
    elif file_import == '../input/edsa-climate-change-belief-analysis-2022/train.csv':
        df = pd.read_csv("../input/edsa-climate-change-belief-analysis-2022/train.csv")
    else:
        error = 'Incorrect File Name!!'
        return error  


    # Remove twitter handles
    df['clean_tweet'] = df['message'].apply(lambda text: remove_handle(text))
    # Remove hashtags
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_hashtags(text))
    # Remove url's
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_url(text))

    # Remove all non alphanumeric characters
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_non_alpha(text))
    # Remove numbers
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_numbers(text))
    # Set to lower case
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: to_lowercase(text))
    # Remove single characters
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_single_characters(text))
    # Remove extra whitespace
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_whitespace(text))
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: remove_stopwords(text))
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: get_tokens(text))
    df['clean_tweet'] = df['clean_tweet'].apply(lambda text: lemmatize_text(text))


    
    df['message'] = df['clean_tweet']

    df = df.drop(['clean_tweet'], 1)
    return df   

In [52]:
processed_df_test = preprocess('test')

In [53]:
processed_df_test

In [ ]:
test1['message'] = df_test['message'].apply(remove_url).apply(remove_handle).apply(remove_hashtags).apply(remove_non_alpha).apply(remove_numbers).apply(to_lowercase).apply(remove_single_characters).apply(remove_whitespace).apply(remove_stopwords).apply(get_tokens).apply(lemmatize_text)
test1['message'].head()

In [ ]:
test1.head()

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tt = TfidfVectorizer(preprocessor=list, tokenizer=list, ngram_range=(1,2), min_df=2, strip_accents = 'ascii', smooth_idf=False)
tt.fit(processed_df['message'])
test_vect = tt.transform(processed_df_test['message'])

In [78]:
test_vect

In [72]:
test_vect.toarray()

# Split the test data into features and labels.

In [73]:
    X = test_vect.toarray()
    y = df['sentiment']

In [74]:
X.shape

In [75]:
y.head()

# Predict on the Test Data

In [79]:
y_pred_test_vect = lr.predict(test_vect)
y_pred_test_vect

# Make a submission

In [81]:
daf=pd.DataFrame(y_pred_test_vect, columns=['sentiment'])
test_vect

In [84]:
output = pd.DataFrame({"tweetid":test['tweetid']})
submission = output.join(daf)
submission.to_csv("submission.csv", index=False)

In [85]:
submission

# Conclusion

I believe the logistic regression model is working fairly well because the F1-macro score on kaggle is 0.71858. Given more time, I would have predicted using other models such as SVM, Decision tree and Random Forest to see which one would predict better results.